In [ ]:
import numpy as np
import cupy as cp
import sys
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from types import SimpleNamespace
import h5py
from matplotlib_scalebar.scalebar import ScaleBar

import warnings
warnings.filterwarnings("ignore", message=f".*peer.*")

sys.path.insert(0, '..')
from utils import *
from rec import Rec
%matplotlib inline

In [ ]:
energy = 19.55
wavelength = 1.24e-09 / energy

detector_pixelsize = 0.55e-6#0.68e-6
focusToDetectorDistance = 2.9#7.060-0.07321

z1 = 0.47
z1c = 0.47
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnifications)
magnifications2 = z1/z1c
voxelsize_code = np.abs(voxelsize/magnifications2)
distancec = (z1-z1c)/(z1c/z1)

n = 2048
ncode = 2048*3//2#2048*3//2
show = True
print(voxelsize,voxelsize_code,magnifications)


In [ ]:
args = SimpleNamespace()
args.ngpus = 1
args.lam = 50

args.n = n
args.ntheta = 1
args.ncode = ncode
args.pad = 0
args.npsi = args.n + 2 * args.pad
args.nq = args.n + 2 * args.pad
args.ex = 32
args.npatch = args.nq + 2 * args.ex
args.nchunk = 1

args.voxelsize = voxelsize
args.wavelength = wavelength
args.distance = distance
args.distancec = distancec
args.eps = 1e-8
args.rho = [1, 2, 0.1]
args.path_out = f"/data/vnikitin/MAXIV/test"
args.show = show

args.niter=4097
args.vis_step=4
args.err_step=4
args.rotation_axis=args.npsi/2

args.theta = np.linspace(0,np.pi,args.ntheta).astype('float32')
# create class
cl_rec = Rec(args)


In [ ]:
import random
import xraylib
import cupyx.scipy.ndimage as ndimage

ill_feature_size = 12e-6
code_thickness = 0.25e-6# 0.55, 1.5, 0.1, 0.25, 2.33,1.08

random.seed(10)
nill = 2*args.ncode
ill_global = cp.zeros([nill,nill],dtype='bool')
ill0 = cp.zeros([nill*nill],dtype='bool')
ill_ids = random.sample(range(0, nill*nill), int(nill*nill*0.55))
ill0[ill_ids] = 1
ill_global = ill0.reshape(nill,nill)

# form codes for simulations
nill = int(args.ncode*voxelsize/np.abs(magnifications2)//(ill_feature_size*2))*2
ill = cp.zeros([nill,nill],dtype='bool')
ill0 = ill_global
ill = ill0[ill0.shape[0]//2-nill//2:ill0.shape[0]//2+(nill)//2,
                ill0.shape[1]//2-nill//2:ill0.shape[1]//2+(nill)//2]#.reshape(nill,nill)

ill = ndimage.zoom(ill,args.ncode/nill,order=0,grid_mode=True,mode='grid-wrap')

delta = 1-xraylib.Refractive_Index_Re('Au',energy,19.3)
beta = xraylib.Refractive_Index_Im('Au',energy,19.3)

print(beta,delta,delta/beta)
thickness = code_thickness/voxelsize # thickness in pixels


# form Transmittance function
Rill = ill*(-delta+1j*beta)*thickness 
Rill=ndimage.rotate(Rill, 45, axes=(2, 1), reshape=False, order=3, mode='reflect',
                   prefilter=True)

# v = cp.arange(-args.ncode//2,args.ncode//2)/2/args.ncode
# [vx,vy] = cp.meshgrid(v,v)
# v = cp.exp(-10*(vx**2+vy**2))
# fill = cp.fft.fftshift(cp.fft.fftn(cp.fft.fftshift(Rill)))
# Rill = cp.fft.fftshift(cp.fft.ifftn(cp.fft.fftshift(fill*v)))
# Rill = Rill.astype('complex64')

code = cp.exp(1j * Rill * voxelsize * 2 * np.pi / wavelength).astype('complex64')

print(f'{voxelsize=}')
print(f'{voxelsize_code=}')
# mshow_polar(code[0],show)



In [ ]:
mshow_polar(code,show)

In [ ]:
# shifts of codes
shifts_code = np.round((np.random.random([args.ntheta, 2]).astype('float32')-0.5)*args.n/4)

In [ ]:
ri = shifts_code.astype('int32')
r = shifts_code-ri

In [ ]:
cdata = np.abs(cl_rec.D(cl_rec.Dc(cl_rec.S(ri,r,code))))**2
mshow(cdata[0],args.show)